In [13]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [14]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [15]:
%run -i ../../zkstats/core.py

In [16]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [17]:
def mode_within(data_array, percent):
    max_sum_freq = 0
    mode = data_array[0]

    for check_val in set(data_array):
        sum_freq = sum(1 for ele in data_array if abs(ele - check_val) <= abs(percent * check_val / 100))

        if sum_freq > max_sum_freq:
            mode = check_val
            max_sum_freq = sum_freq

    return mode

In [18]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

data = json.loads(open(data_path, "r").read())['col_name']
data_tensor = torch.reshape(torch.tensor(data),(1,-1, 1))

create_dummy(data_path, dummy_data_path)
dummy_data = json.loads(open(dummy_data_path, "r").read())['col_name']
dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1,-1,1))

dummy_theory_output = torch.tensor(mode_within(dummy_data,1))
theory_output = torch.tensor(mode_within(data,1))





In [19]:
print("dummy output: ", dummy_theory_output)
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
    def forward(self,X):
        count_equal = torch.sum((torch.abs(X-self.w)<=torch.abs(0.01*self.w)).double())
        result = torch.tensor([torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal for ele in X[0]])
        return (torch.sum(result) == X.size()[1], self.w)
    
verifier_define_calculation(dummy_data_path, ['col_name'],sel_dummy_data_path,verifier_model, verifier_model_path)

dummy output:  tensor(1.2000)


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_1914/3973307746.py:10: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  result = torch.tensor([torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal for ele in X[0]])
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_1914/3973307746.py:10: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  result = torch.tensor([torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal for ele in X[0]])


In [20]:
# prover calculates settings, send to verifier
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        count_equal = torch.sum((torch.abs(X-self.w)<=torch.abs(0.01*self.w)).double())
        result = torch.tensor([torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal for ele in X[0]])
        return (torch.sum(result) == X.size()[1], self.w)

prover_gen_settings(data_path,['col_name'], sel_data_path, prover_model,prover_model_path, 'default', "resources", settings_path)

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_1914/4275092354.py:11: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  result = torch.tensor([torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal for ele in X[0]])


Theory_output:  tensor(77.)
==== Generate & Calibrate Setting ====
scale:  default
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":9,"param_scale":9,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":300,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,9],"model_input_scales":[9],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":[],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_1914/4275092354.py:11: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  result = torch.tensor([torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal for ele in X[0]])


In [21]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0


Time setup: 1.4515161514282227 seconds
Theory output:  tensor(77.)
!@# compiled_model exists? False
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 77.0
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[8148670344449872040, 3850172417927569887, 9386325907200828332, 1904490596485556704], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [7029749938363617618, 11538692499682334571, 8918902057354509017, 3053834031406807957]]], 'proof': '2b4da5fa8dd677f83be0f52255a34e9922f9a066497840548ab0b99178012f70036dbdf7a27d6f852359a2c8c6cbd9142a5c3ac3fb4b0af97a2823feffe67de8080dc52abcfb6d4223f793b1fe0073d29fd6d65fd5d8e399cca97cd8b3652eb323fd7ee4ac45b721ba7a3f1322c74cfcea86f4f7ea09243cc339e44285e5f6e51f622b97fd8e09e215f5c66dc5b64ab47b8f5562a0a64219a53e558f2ecbd3160b6be4209d13a8f5d3b71cc05f9265934a0b75ffe8cf5842ee05f4ebaff1d86c09d34d0505d9b012e88020ee7103261ed6485e687f0ac2d16d58e34d685135bd1e8b0fede9726ba27d570d6cebcb5b34802a2fa8262fe05fa702ed4c6878d7eb183ce71e9f284208c432211481a9ce524bb598cab76cdc5df059e2cd1fd1cd5406174b369bd4b7dc7d3df215c9511fe2ced1f514eba5434a13b2e89505aee5741a4413b2b60759ff93110a3999c98e03dc355edce324a24429417e88a9b9f6be

In [22]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  1
prf instances:  [[[8148670344449872040, 3850172417927569887, 9386325907200828332, 1904490596485556704], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [7029749938363617618, 11538692499682334571, 8918902057354509017, 3053834031406807957]]]
proof boolean:  1.0
proof result 1 : 77.0
verified
